In [1]:
# import libraries (non-cryptogaphic)
import random # to generate phone numbers
import pandas as pd
from functools import reduce
import os

# import libraries (cryptographic)
import cryptography.hazmat.primitives.asymmetric.dh as dh
from cryptography.fernet import Fernet
import hashlib
import sympy
import secrets

# Generate phone numbers and store them in phone_numbers.csv file

In [2]:
random.seed(10) # to ensure same phone numbers generated every time

In [3]:
# class to generate phone numbers for grab and gojek
class PhoneNumberGenerator:
    def __call__(self, count):
        phone_numbers = random.sample(range(80000000,100000000), count)
        return phone_numbers 

class FakePhoneNumberGenerator:
    def __call__(self, count):
        numbers = random.sample(range(10000000,80000000), count) # invalid phone numbers generated
        return numbers
            
    
# class to store numbers to csv
class PhoneNumberStorageManager:
    def __init__(self):
        self.filename = "phone_numbers_padded.csv"
    def __call__(self, gojek_phone_numbers, grab_phone_numbers):
        d = {"gojek": gojek_phone_numbers, 
            "grab": grab_phone_numbers}
        df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()])) # create dataframe
        df.to_csv(self.filename, index = False) # store values to file "phone_numbers_padded.csv"

In [4]:
set_size = 100 # both parties are to exchange sets of a pre-determined size

# generate and store phone numbers
gojek_phone_number_count = 61 # inclusive of phone numbers in common with grab
grab_phone_number_count = 91 # inclusive of phone numbers in common with gojek
common_phone_number_count = 10

# instantiate required classes
phone_number_generator = PhoneNumberGenerator()
fake_phone_number_generator = FakePhoneNumberGenerator()
phone_number_storage_manager = PhoneNumberStorageManager()

# generate phone numbers
phone_numbers = phone_number_generator(gojek_phone_number_count+grab_phone_number_count-common_phone_number_count)
common_phone_numbers = phone_numbers[0:common_phone_number_count]
gojek_phone_numbers = phone_numbers[0:gojek_phone_number_count]
grab_phone_numbers = common_phone_numbers + phone_numbers[gojek_phone_number_count:] 

# generate fake phone numbers
gojek_fake_phone_numbers = fake_phone_number_generator(set_size-gojek_phone_number_count)
grab_fake_phone_numbers = fake_phone_number_generator(set_size-grab_phone_number_count)

# add fake phone numbers to phone numbers
gojek_padded_set = gojek_phone_numbers + gojek_fake_phone_numbers
grab_padded_set = grab_phone_numbers + grab_fake_phone_numbers

# shuffle phone number lists
random.shuffle(gojek_padded_set)
random.shuffle(grab_padded_set)

# write phone numbers (with fake ones) to csv file
phone_number_storage_manager(gojek_padded_set, grab_padded_set)

# Define classes for the necessary for the algorithm

In [5]:
# class to generate numbers required for psi
class NumberGenerator:
    
    def generate_public_parameters(self, size):
        # method to generate p, q and factors of p-1
        p = self.generate_safe_prime(size)
        print(f"p is prime: {sympy.ntheory.isprime(p)}")
        length_of_p = len(bin(p)[2:]) # should be 1024
        print(f"Length of prime modulus, p: {length_of_p}.\nNote: Should be {size}.")
        q = (p-1)//2
        print(f"q is prime: {sympy.ntheory.isprime(q)}")
        factors_pminus1 = [1, 2, q] # since q is prime, 2q only has these 3 factors excluding itself
                                    # 2q = p-1
                                    # factors of p-1 required to compute order of generators (lagrange theorem)
        return p, factors_pminus1

    def generate_safe_prime(self, size):
        # method to generate safe prime for p
        candidate = dh.generate_parameters(2, size).parameter_numbers().p # generate 1024-bit prime number
        while True:
            # the method used from the cryptography already generates a safe prime, this portion is merely to double confirm
            is_safe_prime = sympy.ntheory.isprime((candidate-1)//2) # if safe prime, (candidate-1)/2 is prime
            if (is_safe_prime):
                break
            else:
                candidate = dh.generate_parameters(2, size).parameter_numbers().p
                print(candidate)
 
        return candidate
    
    
    def generate_random_number(self, size):
        # method to generate client's secret
        return secrets.randbits(size)
            
# class to inspect values          
class NumberInspector:
    
    def check_is_primitive_generator(self, candidate, factors_divisorminus1, divisor): # note: factors should be the factors of divisor-1
        
        # apply lagrange theorem
        for possible_order in factors_divisorminus1: # check congruence for all factors (factors is exclusive of p-1 itself)
            result = pow(candidate, possible_order, divisor) # fast modular exponentiation implemented in pow(x,y,z)
                                                             # python integers have arbitrary precisions, 
                                                             # no overflow would occur if operations done in pure python
            if (result == 1):
                return False # if candidate exponent any of the factors (1,2,q) congruent to 1modp, candidate is not a primitive
                             # generator
        return True # order of candidate == p-1, therefore candidate is a primitive generator

    
class StorageManager:
    
    def store_data(self, filename, data):
        df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe
        df.to_csv(filename, index=False) # store data to file with filename


# Create psi client class

In [6]:
# client class (both grab and gojek are clients communicating directly with each other)
class Client:
    def __init__(self, name, other_party_name, private_key_size, phone_numbers, p, factors_pminus1, fernet_key):
        
        self.number_inspector = NumberInspector()
        self.number_generator = NumberGenerator()
        
        # for asymmetric encryption
        self.private_key = self.number_generator.generate_random_number(private_key_size)
        self.my_set = phone_numbers
        self.p = p # prime modulus
        self.factors_pminus1 = factors_pminus1 # to calculate order of hashed phone numbers, 
                                               # since algorithm requires them to be primitive generators
        
        # for symmetric encryption
        self.fernet_key = fernet_key
        self.f = Fernet(fernet_key)
        
        # values to track for psi algorithm
        self.my_hashed_set = None # h(x)
        self.my_self_encrypted_set = None # (h(x)^(my_secret))modp
        self.my_encrypted_set = None # (h(x)^(my_secret)(other_party_secret))modp
        self.other_party_encrypted_set = None # (h(y)^(my_secret)(other_party_secret))modp
        self.common_values = None # common phone numbers
        
        
        # create datafile for communication with another party
        
        # content to store in file
        # only need to share self_encrypted_values and other_party_encrypted_values
        # common_values to ensure both calculate the same intersection
        self.my_dict = {
            'my_self_encrypted_set': None, 
            'other_party_encrypted_set': None,
            'common_values': None
        }
        
        # filenames
        self.name = name
        self.filename = name + "_data_v1.2.csv"
        self.other_party_name = other_party_name
        self.other_party_filename = other_party_name + "_data_v1.2.csv"
        
        # create file
        self.storage_manager = StorageManager()
        self.storage_manager.store_data(self.filename, self.my_dict)

    def hash_to_primitive_root_modulo_p(self, element): 
        # method to hash phone numbers to primitive root modulo p i.e. primitive generator

        endian = "big"
        element = element.to_bytes(4, endian)
        hash_hex = hashlib.sha256(element).hexdigest() # sha3_256
        hash_int = int(hash_hex, 16)
        while True:
            # repeatedly hash until primitive root modulo p is obtained
            is_primitive_generator = self.number_inspector.check_is_primitive_generator(
                hash_int, self.factors_pminus1, self.p
            )
            if (is_primitive_generator):
                break
            else:
                hash_int = hash_int.to_bytes(32, endian)
                hash_hex = hashlib.sha256(hash_int).hexdigest()
                hash_int = int(hash_hex, 16)
                
        return hash_int
    
    def modular_exponentation(self, element):
        # compute (element^(private_key))modp
    
        return pow(element, self.private_key, self.p)
    
    def hash_set(self):
        # hash all phone numbers in my set to primitive root modulo p, one by one
        
        self.my_hashed_set = []
        
        for element in self.my_set:
            hashed_value = self.hash_to_primitive_root_modulo_p(element)
            self.my_hashed_set.append(hashed_value)
            
    def encrypt_set(self, is_other_party):
        # encrypt all elements in a given set using private_key, one by one
        
        # two scenarios to consider
        # one: encrypt set sent by the other party
        if (is_other_party):
            decrypted_other_party_set = self.receive_data("my_self_encrypted_set")
            other_party_set_int = []
            for element_string in decrypted_other_party_set:
                other_party_set_int.append(int(element_string))
            set_to_encrypt = other_party_set_int
        # two: encrypt my own set
        else:
            set_to_encrypt = self.my_hashed_set
        
        # encrypt values in given set, one by one
        encrypted_values = []
        for element in set_to_encrypt:
            encrypted_value = self.modular_exponentation(element)
            encrypted_values.append(encrypted_value)
            
        # assign the encrypted set to the correct variable
        # update csv file used for communication
        if (is_other_party):
            self.other_party_encrypted_set = encrypted_values
            self.send_data(encrypted_values, "other_party_encrypted_set")
        else:
            self.my_self_encrypted_set = encrypted_values
            self.send_data(encrypted_values, "my_self_encrypted_set")
                    
        
    def get_intersection(self):
        
        # get intersection
        
        my_encrypted_set = self.receive_data("other_party_encrypted_set") # read my encrypted set from the other party's file
        my_encrypted_set_int = []
        
        # convert read values to integer
        for element in my_encrypted_set:
            my_encrypted_set_int.append(int(element))
            
        # assign to correct variable
        self.my_encrypted_set = my_encrypted_set_int
        
        # get intersection
        encrypted_common_values = set(self.my_encrypted_set).intersection(self.other_party_encrypted_set)
        index_of_common_values = []
        
        # find the index of the elements in the intersection in my_encrypted_set
        for element in encrypted_common_values:
            index_of_common_values.append(self.my_encrypted_set.index(element))
            
        self.common_values = []
        
        # find the values in my own set corresponding to the index of the elements in the intersection
        for index in index_of_common_values:
            potential_phone_number = self.my_set[index]
            
            if ((potential_phone_number-79999999)>0): # remove any potential fake numbers that intersect
                self.common_values.append(potential_phone_number)
        
        # update csv file for communication
        self.my_dict["common_values"] = self.common_values
        self.send_data(self.common_values, "common_values")
    
    def encrypt_data(self, plaintext):
        # encrypt data with Fernet
        
        endian = "big"
        element = plaintext.to_bytes(128, endian) # 1024 bits == 128 bytes
        cipher_text = self.f.encrypt(element)
        return cipher_text
    
    def decrypt_data(self, ciphertext):
        # decrypt data encrypted by Fernet
        
        ciphertext_bytes = ciphertext.encode('utf-8')[2:-1] # convert from string back to bytes
        endian = "big"
        element_in_bytes = self.f.decrypt(ciphertext_bytes)
        plaintext = int.from_bytes(element_in_bytes, endian)
        return plaintext

    def send_data(self, data_to_send, column_name):
        # send data means writing to file. encrypt data with Fernet
        
        # encrypt data
        encrypted_data_to_send = []
        for element in data_to_send:
            encrypted_element = self.encrypt_data(element)
            encrypted_data_to_send.append(encrypted_element)
            
        # send data
        self.my_dict[column_name] = encrypted_data_to_send
        self.storage_manager.store_data(self.filename, self.my_dict)
        
        
    def receive_data(self, column_name):
        # receive data means reading from file (my file). decrypt data encrypted by Fernet
        
        # receive data
        encrypted_data = self.get_other_party_data()[column_name].to_list()
        
        # decrypt data
        decrypted_data = []
        for element in encrypted_data:
            if type(element) is float: # remove NaN
                continue
            decrypted_element = self.decrypt_data(element)
            decrypted_data.append(decrypted_element)
        print(f"number of elements in {self.other_party_name};{column_name}: {len(decrypted_data)}.")
        return decrypted_data
    
    def get_my_data(self):
        # read my file as dataframe (other party's file)
        
        return pd.read_csv(self.filename)
            
    def get_other_party_data(self):
        # read other party's file as dataframe
        
        return pd.read_csv(self.other_party_filename)
    
    


# Initialize context

In [7]:
# assign pre-determined variables for psi
key_size = 1024 # both private keys and large prime

# create key for symmetric key cryptography
fernet_key = Fernet.generate_key()

# create public parameters required for psi
number_generator = NumberGenerator()
p, factors_pminus1 = number_generator.generate_public_parameters(key_size)

# create clients
grab = Client("grab", "gojek", key_size, grab_padded_set, p, factors_pminus1, fernet_key)
gojek = Client("gojek", "grab", key_size, gojek_padded_set, p, factors_pminus1, fernet_key)



p is prime: True
Length of prime modulus, p: 1024.
Note: Should be 1024.
q is prime: True


C:\Users\jiaji\AppData\Local\Temp/ipykernel_15924/2734410534.py:55: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe


# Get intersection

## Step 1: Hash phone numbers

In [8]:
# clients hash their own set
grab.hash_set()
gojek.hash_set()

### Clients' status after step 1
Note: hashed set is not stored in the file as it is not meant to be shared with the other party hence, files' status after step 1 is not shown

In [9]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set
0,3444554169003369415400187425444370377909224933...,4373944936197365140068497953964796764903962679...
1,7565557829042128713146902939719761430738184184...,8518371235178301712300445095786395503442694033...
2,4655239225546648301658825221353687888127750153...,2604059332946975929382400105366344085452568333...
3,7165927783480481448708501993505812863126714098...,2835741223782226842638506406959946569416451500...
4,2701445021582149789903076929700056578715457912...,4320682999770379726688099447684191745405506617...
...,...,...
95,4464107689706471902374892139671036333433126606...,2064846653753480132804612906272453669387169259...
96,1516234710201293820270438455976630785218809049...,8362019212874245816279708570292780730782352063...
97,4270367225745395469793884445520741841580153424...,3948222474783499474687611776797431125471701320...
98,5888437846314481919580418714275445066774752189...,7018556158959834041117086565402060628499966280...


## Step 2: encrypt hashed set with own private key

In [10]:
# clients self encrypt hashed set
grab.encrypt_set(False) # set is_other_party to false to encrypt own hashed set
gojek.encrypt_set(False)

C:\Users\jiaji\AppData\Local\Temp/ipykernel_15924/2734410534.py:55: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe


### Clients' status after step 2 (value of variables in client)

In [11]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set
0,3444554169003369415400187425444370377909224933...,4373944936197365140068497953964796764903962679...,8187474701304423266974143713908105140320000088...,7514936027236315344897179514688652153863641524...
1,7565557829042128713146902939719761430738184184...,8518371235178301712300445095786395503442694033...,4046914014279726218715416480445576320458145676...,1104638765095112396958481231454906029458849529...
2,4655239225546648301658825221353687888127750153...,2604059332946975929382400105366344085452568333...,7242496355022601657819988902609271214468467779...,1159546688050557193034564612756279562240838681...
3,7165927783480481448708501993505812863126714098...,2835741223782226842638506406959946569416451500...,8200923328160401898800582870090781918345046779...,1002666411520646847742189777973491105102644886...
4,2701445021582149789903076929700056578715457912...,4320682999770379726688099447684191745405506617...,5384686795144887474807598417126967049347624667...,4643954305118295589390903972421186395643079806...
...,...,...,...,...
95,4464107689706471902374892139671036333433126606...,2064846653753480132804612906272453669387169259...,1013870224445208690018120456554223155073570627...,6685443834370859267579142392833022594187744976...
96,1516234710201293820270438455976630785218809049...,8362019212874245816279708570292780730782352063...,1109857918321247424255392157808813372975866341...,2859292490170269069637392052641078258212627594...
97,4270367225745395469793884445520741841580153424...,3948222474783499474687611776797431125471701320...,3710364077259870631700380266012130594334485589...,2082878913103926582467019386760129323365578767...
98,5888437846314481919580418714275445066774752189...,7018556158959834041117086565402060628499966280...,1002374013867708277195507365683411758096014321...,1081664733027808405492946889431548381542552304...


### Files' status after step 2 (value of variables in file - clients' variables encrypted with Fernet)

In [12]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek


gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhofN5cAebo_znZiSQfwE2UFEyvdNkAxfGsh7P...,NaN,NaN
1,b'gAAAAABhofN5GDOGSXjjPreihtz3jZmcjIqCMOAevWI5...,NaN,NaN
2,b'gAAAAABhofN5JWMda6aKEV1YuQxzZlxrfb7Wm8ehPjni...,NaN,NaN
3,b'gAAAAABhofN5Ep2ld9cj0HGzKpScX-t-_68-ITBsm8HL...,NaN,NaN
4,b'gAAAAABhofN52Z7S71Ye48LYs574ZgU0DMkQ_Z86-yow...,NaN,NaN
...,...,...,...
95,b'gAAAAABhofN5U7UaQHUSY0LXSWWOVKQP-ey624gFAn28...,NaN,NaN
96,b'gAAAAABhofN5CmVfvFVKVgEXqJ3VPtomhftwCElgkELT...,NaN,NaN
97,b'gAAAAABhofN5-jZWI3SCVjs6ATTZKBv710BbF4rplmug...,NaN,NaN
98,b'gAAAAABhofN5q4qcVWGhFKLnMS6_l-akCeNo7Uoub-Oo...,NaN,NaN


In [13]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhofN5CpI2lYdwC0zwGWeNUXQabTaLexX43eoY...,NaN,NaN
1,b'gAAAAABhofN5z4eEGt2M441DKBjF3asmr3dBIxEMKsBJ...,NaN,NaN
2,b'gAAAAABhofN5ba86H4dY0yBz815cVEH_kkfTgPkhXY3d...,NaN,NaN
3,b'gAAAAABhofN5z5bX9GLhkghLlJEln2XbnYfmUeK3Ek5b...,NaN,NaN
4,b'gAAAAABhofN5AhdyTMNB74j239Gvqr4STuq3PSrbOXVR...,NaN,NaN
...,...,...,...
95,b'gAAAAABhofN51GfBuDAdEfdd0DX4WMhR2fAugXcLTo_v...,NaN,NaN
96,b'gAAAAABhofN5kBJI0B1J5w3iDGhbSbdEQ0QxAE1kv7Qn...,NaN,NaN
97,b'gAAAAABhofN5TtV9eHmV053YvDZCpGm46eT_RUmMdiQx...,NaN,NaN
98,b'gAAAAABhofN5o6maXKPgH4yx3B7o8Ee6ZXxvAamZ4URY...,NaN,NaN


## Step 3: encrypt other party's self-encrypted set with own private key


In [14]:
# clients encrypt other party's self encrypted set
grab.encrypt_set(True) # set is_other_party to true
gojek.encrypt_set(True)

number of elements in gojek;my_self_encrypted_set: 100.


C:\Users\jiaji\AppData\Local\Temp/ipykernel_15924/2734410534.py:55: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe


number of elements in grab;my_self_encrypted_set: 100.


### Clients' status after step 3

In [15]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set,
    "gojek encrypted set": grab.other_party_encrypted_set,
    "grab encrypted set": gojek.other_party_encrypted_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set,gojek encrypted set,grab encrypted set
0,3444554169003369415400187425444370377909224933...,4373944936197365140068497953964796764903962679...,8187474701304423266974143713908105140320000088...,7514936027236315344897179514688652153863641524...,8854838848257798603077541640012422101891444898...,8023728087263571659585275852445525420227798030...
1,7565557829042128713146902939719761430738184184...,8518371235178301712300445095786395503442694033...,4046914014279726218715416480445576320458145676...,1104638765095112396958481231454906029458849529...,1063171329072069596620549965607214058358966596...,1529561356641365070223543272377648712410277208...
2,4655239225546648301658825221353687888127750153...,2604059332946975929382400105366344085452568333...,7242496355022601657819988902609271214468467779...,1159546688050557193034564612756279562240838681...,6276695769656984984273746883361111193625504895...,8231953613760818198657027255709274848196045213...
3,7165927783480481448708501993505812863126714098...,2835741223782226842638506406959946569416451500...,8200923328160401898800582870090781918345046779...,1002666411520646847742189777973491105102644886...,1634367227182502225237692608524268527113904579...,4872897770334820130984789578645727410677232717...
4,2701445021582149789903076929700056578715457912...,4320682999770379726688099447684191745405506617...,5384686795144887474807598417126967049347624667...,4643954305118295589390903972421186395643079806...,1109537573587154799685738162785176648144482161...,7276958489083220096131337136217293268394147327...
...,...,...,...,...,...,...
95,4464107689706471902374892139671036333433126606...,2064846653753480132804612906272453669387169259...,1013870224445208690018120456554223155073570627...,6685443834370859267579142392833022594187744976...,1229472990757717282364586481302934017524494011...,1298665735532974734052057242476721353529749127...
96,1516234710201293820270438455976630785218809049...,8362019212874245816279708570292780730782352063...,1109857918321247424255392157808813372975866341...,2859292490170269069637392052641078258212627594...,6337488188969103524992864385784992007297470943...,1179255848082101247877357600722504923860031222...
97,4270367225745395469793884445520741841580153424...,3948222474783499474687611776797431125471701320...,3710364077259870631700380266012130594334485589...,2082878913103926582467019386760129323365578767...,1022511939954103984912003645271943128885773922...,4919196638036520620841495535492796523381965434...
98,5888437846314481919580418714275445066774752189...,7018556158959834041117086565402060628499966280...,1002374013867708277195507365683411758096014321...,1081664733027808405492946889431548381542552304...,8343622018244084312210167797727427573434401296...,1118970412753577110848971288685289097643070812...


### Files' status after step 3

In [16]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek



gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhofN5cAebo_znZiSQfwE2UFEyvdNkAxfGsh7P...,b'gAAAAABhofN6cIBTLHY_fGUADYqpD5HVbvCTnWzLABEh...,NaN
1,b'gAAAAABhofN5GDOGSXjjPreihtz3jZmcjIqCMOAevWI5...,b'gAAAAABhofN6F0MuiSIfhNFIywc90wu1w-t9-r8mE9HC...,NaN
2,b'gAAAAABhofN5JWMda6aKEV1YuQxzZlxrfb7Wm8ehPjni...,b'gAAAAABhofN6Z2IFEavpQOlH8gN65NSDiENdWqoFaw3x...,NaN
3,b'gAAAAABhofN5Ep2ld9cj0HGzKpScX-t-_68-ITBsm8HL...,b'gAAAAABhofN6aUhQs7Ize3Nc_p06i_HY7M3i1il9A8bL...,NaN
4,b'gAAAAABhofN52Z7S71Ye48LYs574ZgU0DMkQ_Z86-yow...,b'gAAAAABhofN6cpBsPh1zrTFodJnG0IRWjjvAtMECzogv...,NaN
...,...,...,...
95,b'gAAAAABhofN5U7UaQHUSY0LXSWWOVKQP-ey624gFAn28...,b'gAAAAABhofN6zeBby2FWbx18QK4EuBtQuNGopQqWcE0V...,NaN
96,b'gAAAAABhofN5CmVfvFVKVgEXqJ3VPtomhftwCElgkELT...,b'gAAAAABhofN68zqobJj-lNMIPk-DeIbwP1pmRXf9TmlK...,NaN
97,b'gAAAAABhofN5-jZWI3SCVjs6ATTZKBv710BbF4rplmug...,b'gAAAAABhofN6cawM3mpcnDz0ONiaiAbvXaaT9sah84u6...,NaN
98,b'gAAAAABhofN5q4qcVWGhFKLnMS6_l-akCeNo7Uoub-Oo...,b'gAAAAABhofN6na5Ajn48Z2k8DUSCzslRajpIkK7wTfAd...,NaN


In [17]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhofN5CpI2lYdwC0zwGWeNUXQabTaLexX43eoY...,b'gAAAAABhofN62vjs_cyf024Ttzzcj__Zgiv3WvnwdApR...,NaN
1,b'gAAAAABhofN5z4eEGt2M441DKBjF3asmr3dBIxEMKsBJ...,b'gAAAAABhofN6FswUoGtciQltq5djMX6pMLXnSk2j9WAA...,NaN
2,b'gAAAAABhofN5ba86H4dY0yBz815cVEH_kkfTgPkhXY3d...,b'gAAAAABhofN6IdDKQid7_NvSplwJsiYaKTWknYP3lgWV...,NaN
3,b'gAAAAABhofN5z5bX9GLhkghLlJEln2XbnYfmUeK3Ek5b...,b'gAAAAABhofN68VbogGxvsmcLskt_OcWMVUcVtAIT66S9...,NaN
4,b'gAAAAABhofN5AhdyTMNB74j239Gvqr4STuq3PSrbOXVR...,b'gAAAAABhofN6jCsnTw0VORCXuQu4RM2T2iwQXdaVNwR2...,NaN
...,...,...,...
95,b'gAAAAABhofN51GfBuDAdEfdd0DX4WMhR2fAugXcLTo_v...,b'gAAAAABhofN6H_u9uSkEXLnfwalCqfHlwGmq-onTdPu_...,NaN
96,b'gAAAAABhofN5kBJI0B1J5w3iDGhbSbdEQ0QxAE1kv7Qn...,b'gAAAAABhofN6urcD9GbEKYXQIjea1oCB4MUGUhIIx0-9...,NaN
97,b'gAAAAABhofN5TtV9eHmV053YvDZCpGm46eT_RUmMdiQx...,b'gAAAAABhofN6DENY05pneTsKsw2Y6G08NFPnEO7gRxpI...,NaN
98,b'gAAAAABhofN5o6maXKPgH4yx3B7o8Ee6ZXxvAamZ4URY...,b'gAAAAABhofN6yOgXgKYOGS50B_NCY9v_ZOdQsyfVQr05...,NaN


## Step 4: find intersection


In [18]:
# clients find intersection
grab.get_intersection()
gojek.get_intersection()

number of elements in gojek;other_party_encrypted_set: 100.
number of elements in grab;other_party_encrypted_set: 100.


### Clients' status after step 4:

In [19]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set,
    "gojek encrypted set": grab.other_party_encrypted_set,
    "grab encrypted set": gojek.other_party_encrypted_set,
    "gojek found intersection": gojek.common_values,
    "grab found intersection:": grab.common_values}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set,gojek encrypted set,grab encrypted set,gojek found intersection,grab found intersection:
0,3444554169003369415400187425444370377909224933...,4373944936197365140068497953964796764903962679...,8187474701304423266974143713908105140320000088...,7514936027236315344897179514688652153863641524...,8854838848257798603077541640012422101891444898...,8023728087263571659585275852445525420227798030...,89312048.0,89312048.0
1,7565557829042128713146902939719761430738184184...,8518371235178301712300445095786395503442694033...,4046914014279726218715416480445576320458145676...,1104638765095112396958481231454906029458849529...,1063171329072069596620549965607214058358966596...,1529561356641365070223543272377648712410277208...,99173089.0,99173089.0
2,4655239225546648301658825221353687888127750153...,2604059332946975929382400105366344085452568333...,7242496355022601657819988902609271214468467779...,1159546688050557193034564612756279562240838681...,6276695769656984984273746883361111193625504895...,8231953613760818198657027255709274848196045213...,81093373.0,86915509.0
3,7165927783480481448708501993505812863126714098...,2835741223782226842638506406959946569416451500...,8200923328160401898800582870090781918345046779...,1002666411520646847742189777973491105102644886...,1634367227182502225237692608524268527113904579...,4872897770334820130984789578645727410677232717...,86915509.0,81093373.0
4,2701445021582149789903076929700056578715457912...,4320682999770379726688099447684191745405506617...,5384686795144887474807598417126967049347624667...,4643954305118295589390903972421186395643079806...,1109537573587154799685738162785176648144482161...,7276958489083220096131337136217293268394147327...,94391128.0,94391128.0
...,...,...,...,...,...,...,...,...
95,4464107689706471902374892139671036333433126606...,2064846653753480132804612906272453669387169259...,1013870224445208690018120456554223155073570627...,6685443834370859267579142392833022594187744976...,1229472990757717282364586481302934017524494011...,1298665735532974734052057242476721353529749127...,NaN,NaN
96,1516234710201293820270438455976630785218809049...,8362019212874245816279708570292780730782352063...,1109857918321247424255392157808813372975866341...,2859292490170269069637392052641078258212627594...,6337488188969103524992864385784992007297470943...,1179255848082101247877357600722504923860031222...,NaN,NaN
97,4270367225745395469793884445520741841580153424...,3948222474783499474687611776797431125471701320...,3710364077259870631700380266012130594334485589...,2082878913103926582467019386760129323365578767...,1022511939954103984912003645271943128885773922...,4919196638036520620841495535492796523381965434...,NaN,NaN
98,5888437846314481919580418714275445066774752189...,7018556158959834041117086565402060628499966280...,1002374013867708277195507365683411758096014321...,1081664733027808405492946889431548381542552304...,8343622018244084312210167797727427573434401296...,1118970412753577110848971288685289097643070812...,NaN,NaN


Note: Last 2 columns, unlike the rest of the columns, do not have a one-to-one mapping with other values belonging to the same row i.e. values in the last 2 columns do not have any relation to the other values in the same row as it.

### Files' status after step 4

In [20]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek


gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhofN5cAebo_znZiSQfwE2UFEyvdNkAxfGsh7P...,b'gAAAAABhofN6cIBTLHY_fGUADYqpD5HVbvCTnWzLABEh...,b'gAAAAABhofN6IPR4Mur4oIEfDKSTZ5kzAo9V9qIjPCPz...
1,b'gAAAAABhofN5GDOGSXjjPreihtz3jZmcjIqCMOAevWI5...,b'gAAAAABhofN6F0MuiSIfhNFIywc90wu1w-t9-r8mE9HC...,b'gAAAAABhofN6A0rJLA3f61KZ7yXkesfnLWsXe2HRZBz_...
2,b'gAAAAABhofN5JWMda6aKEV1YuQxzZlxrfb7Wm8ehPjni...,b'gAAAAABhofN6Z2IFEavpQOlH8gN65NSDiENdWqoFaw3x...,b'gAAAAABhofN6rcDs-UC6ILG1rMcilLmpIubwFyOWrkk-...
3,b'gAAAAABhofN5Ep2ld9cj0HGzKpScX-t-_68-ITBsm8HL...,b'gAAAAABhofN6aUhQs7Ize3Nc_p06i_HY7M3i1il9A8bL...,b'gAAAAABhofN6BUAkls9IdZmBhdE-9V5RRj4wSqgmo-n8...
4,b'gAAAAABhofN52Z7S71Ye48LYs574ZgU0DMkQ_Z86-yow...,b'gAAAAABhofN6cpBsPh1zrTFodJnG0IRWjjvAtMECzogv...,b'gAAAAABhofN6-EZCyP3UTHz-lM6myolR7JvNJggoWiTU...
...,...,...,...
95,b'gAAAAABhofN5U7UaQHUSY0LXSWWOVKQP-ey624gFAn28...,b'gAAAAABhofN6zeBby2FWbx18QK4EuBtQuNGopQqWcE0V...,NaN
96,b'gAAAAABhofN5CmVfvFVKVgEXqJ3VPtomhftwCElgkELT...,b'gAAAAABhofN68zqobJj-lNMIPk-DeIbwP1pmRXf9TmlK...,NaN
97,b'gAAAAABhofN5-jZWI3SCVjs6ATTZKBv710BbF4rplmug...,b'gAAAAABhofN6cawM3mpcnDz0ONiaiAbvXaaT9sah84u6...,NaN
98,b'gAAAAABhofN5q4qcVWGhFKLnMS6_l-akCeNo7Uoub-Oo...,b'gAAAAABhofN6na5Ajn48Z2k8DUSCzslRajpIkK7wTfAd...,NaN


In [21]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhofN5CpI2lYdwC0zwGWeNUXQabTaLexX43eoY...,b'gAAAAABhofN62vjs_cyf024Ttzzcj__Zgiv3WvnwdApR...,b'gAAAAABhofN6H3N_00NW7FX2n_JvAaFaPYmpfAeeizNK...
1,b'gAAAAABhofN5z4eEGt2M441DKBjF3asmr3dBIxEMKsBJ...,b'gAAAAABhofN6FswUoGtciQltq5djMX6pMLXnSk2j9WAA...,b'gAAAAABhofN6wunmueiaZJjkwERoVh59-GmKEv43tThN...
2,b'gAAAAABhofN5ba86H4dY0yBz815cVEH_kkfTgPkhXY3d...,b'gAAAAABhofN6IdDKQid7_NvSplwJsiYaKTWknYP3lgWV...,b'gAAAAABhofN6WdkLsN31_u2XXNbzWrUYbYi8rTPEjz-4...
3,b'gAAAAABhofN5z5bX9GLhkghLlJEln2XbnYfmUeK3Ek5b...,b'gAAAAABhofN68VbogGxvsmcLskt_OcWMVUcVtAIT66S9...,b'gAAAAABhofN6lsJSnA5dIrt18KqNACT2cXSaR7DmO6BK...
4,b'gAAAAABhofN5AhdyTMNB74j239Gvqr4STuq3PSrbOXVR...,b'gAAAAABhofN6jCsnTw0VORCXuQu4RM2T2iwQXdaVNwR2...,b'gAAAAABhofN632XDO2v_SAM8p2_KJKa6KoVZzMAmo2bA...
...,...,...,...
95,b'gAAAAABhofN51GfBuDAdEfdd0DX4WMhR2fAugXcLTo_v...,b'gAAAAABhofN6H_u9uSkEXLnfwalCqfHlwGmq-onTdPu_...,NaN
96,b'gAAAAABhofN5kBJI0B1J5w3iDGhbSbdEQ0QxAE1kv7Qn...,b'gAAAAABhofN6urcD9GbEKYXQIjea1oCB4MUGUhIIx0-9...,NaN
97,b'gAAAAABhofN5TtV9eHmV053YvDZCpGm46eT_RUmMdiQx...,b'gAAAAABhofN6DENY05pneTsKsw2Y6G08NFPnEO7gRxpI...,NaN
98,b'gAAAAABhofN5o6maXKPgH4yx3B7o8Ee6ZXxvAamZ4URY...,b'gAAAAABhofN6yOgXgKYOGS50B_NCY9v_ZOdQsyfVQr05...,NaN


# Check results

In [22]:
# get intersection found by the two parties
gojek_found_intersection = gojek.common_values
grab_found_intersection = grab.common_values


# sort numbers for easier comparison
gojek_found_intersection.sort()
grab_found_intersection.sort()
common_phone_numbers.sort()

# summarize them in a dataframe
d = {"actual": common_phone_numbers,
    "gojek": gojek_found_intersection,
    "grab": grab_found_intersection}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,actual,gojek,grab
0,80497694,80497694,80497694
1,81093373,81093373,81093373
2,86915509,86915509,86915509
3,89312048,89312048,89312048
4,94391128,94391128,94391128
5,95521626,95521626,95521626
6,96192082,96192082,96192082
7,96485172,96485172,96485172
8,99173089,99173089,99173089
9,99397525,99397525,99397525
